### Datenanreicherung – Artikellinks extrahieren

Dieses Notebook extrahiert Headlines und Links aus den zur Verfügung gestellten HTML-Dateien ausgewählter Medien, die eines der definierten Keywords BVG, HVV (inkl. VHH, Hochbahn)oder MVG enthalten.

Die wichtigsten Schritte:
- Extraktion von Links und Headlines aus HTML-Dateien auf Basis der vorliegenden Metadaten
- Speicherung der Ergebnisse als CSV-Dateien
- Protokollierung potenzieller Verarbeitungsfehler zur Nachvollziehbarkeit

Dieses Skript setzt voraus, dass die Daten bereits entpackt und geprüft wurden (siehe: 01_entpacken_und_rohdatenanalyse.ipynb)

#### 1. Import benötigte Pakete

In [1]:
# Standard
import os # Dateipfaden
import pandas as pd # Tabellenverarbeitung (DataFrames)

# HTML-Verarbeitung 
from bs4 import BeautifulSoup # HTML analysieren
from glob import glob # Dateisuche

In [2]:
# Pfade
# Projektverzeichnis
PROJECT_ROOT = r"D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping"

# Input-Pfade
INPUT_PATH = os.path.join(PROJECT_ROOT, "input", "raw") 
DATA_LAKE_PATH = os.path.join(INPUT_PATH, "data-lake") 
STORAGE_PATH = DATA_LAKE_PATH

# Output-Pfad
OUTPUT_PATH = os.path.join(PROJECT_ROOT, "output") 
CSV_PATH = os.path.join(OUTPUT_PATH, "artikel_links_headlines.csv")
FAILED_PATH = os.path.join(OUTPUT_PATH, "artikel_links_failed.csv")
LOG_PATH = os.path.join(OUTPUT_PATH, "artikel_links_log.csv")

#### 2. Funktionen zur Verarbeitung definieren

In [3]:
# Parameter definieren
# Auswahl an Medien
MEDIEN = [
    "dlf", "tagesschau",
    "handelsblatt", "wiwo", "mm", "boerse",
    "sz", "zeit", "faz", "taz", "welt", "spiegel", "stern",
    "abendblatt", "berliner", "tagesspiegel",
    "ntv", "pioneer", "dw-de",
    "heise", "golem", "netzpolitik", "t3n"
]

# Keywords: Filterung von Artikellinks
KEYWORDS = ["bvg", "mvg", "hvv", "vhh", "hochbahn"]

In [4]:
# Funktion: Links und Headlines aus HTML-Datei anhand der Metadatenzeile extrahieren
def extract_links_and_headlines(newspaper):
  
    # Dateiname, Pfad, Encoding und Basis-URL aus den Metadaten
    filename = os.path.basename(newspaper["file_name"])
    full_path = os.path.join(DATA_LAKE_PATH, filename)
    encoding = newspaper["encoding"].lower()
    url_base = newspaper["final_url"].strip()  

    # HTML-Datei einlesen
    try:
        with open(full_path, "r", encoding=encoding) as f:
            html = f.read()
    except Exception as e:
        print(f"[ERROR] Fehler beim Lesen von {filename}: {e}")
        return pd.DataFrame()

    # HTML parsen mit BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Ergebnisliste vorbereiten
    records = []

    # Link-Tags im HTML durchsuchen
    for link_tag in soup.find_all("a", href=True):
        href = link_tag["href"]
        # Sichtbarer Text innerhalb des Links holen
        text = link_tag.get_text(strip=True)
        # Beide Texte für spätere Suche Keywords kombinieren
        combined = (href + " " + text).lower()

        # Weiterverarbeiten nur, wenn mindestens ein Keyword vorkommt
        if any(kw in combined for kw in KEYWORDS):
            # Vollständige URL rekonstruieren
            if href.startswith("/"):
                full_url = url_base.rstrip("/") + href
            elif href.startswith("http"):
                full_url = href
            else:
                continue  

            # Nur aufnehmen, wenn headline existiert
            if text:
                records.append({
                    "filename": filename, # Dateilname
                    "source": newspaper["name"], # Medium
                    "date": newspaper["date"], # Veröffentlichungsdatum
                    "url": full_url, # http-Adresse
                    "headline": text # Headline
                })

    # Ergebnis als DataFrame speichern
    return pd.DataFrame(records) 

#### 3. Iteratives Anwenden der Verarbeitungsfunktionen

In [5]:
# Links und Headlines aus HTML-Datei anhand der Metadatenzeile extrahieren

# Ergebnis-Listen
results = []
log_list = []
failing_list = []

# Alle Metadaten-CSV-Dateien durchlaufen
csv_files = sorted(glob(os.path.join(STORAGE_PATH, "*.csv")))

for csv_path in csv_files:
    try:        
        df = pd.read_csv(csv_path)

        # Filter nur ausgewählte Medien
        df = df[df["name"].isin(MEDIEN)]

        if df.empty:
            continue

        # Für jede Zeile (HTML-Datei) Links extrahieren
        for _, row in df.iterrows():
            try:  
                result = extract_links_and_headlines(row)
                results.append(result)
                log_list.append({
                    "filename": row["file_name"],
                    "source": row["name"],
                    "date": row["date"],
                    "num_links": len(result)
                })
            except Exception as e:
                failing_list.append({
                    "filename": row["file_name"],
                    "source": row["name"],
                    "error": str(e)
                })

    except Exception as e:
        print(f"[FEHLER] Fehler beim Einlesen von {csv_path}: {e}")

In [6]:
# Ergebnisse zusammenführen und anzeigen
if results:
    df_artikel = pd.concat(results, ignore_index=True)
    print("[INFO] Anzahl gefundener Links insgesamt:", len(df_artikel))

    # Ergebnis als CSV-Datei exportieren
    df_artikel.to_csv(CSV_PATH, index=False)
    print(f"[INFO] Verarbeitete Metadaten-Dateien: {len(csv_files)}")
    print(f"[INFO] Links gespeichert unter: {CSV_PATH}")
else:
    print("[WARNUNG] Keine Artikellinks gefunden.")

[INFO] Anzahl gefundener Links insgesamt: 5593
[INFO] Verarbeitete Metadaten-Dateien: 1490
[INFO] Links gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikel_links_headlines.csv


#### 4. Speicherung

In [7]:
# Fehlerliste in DataFrame umwandeln und als CSV-Datei exportieren
pd.DataFrame(failing_list).to_csv(FAILED_PATH, index=False)
print(f"[INFO] Fehlerhafte HTML-Dateien: {len(failing_list)}")
print(f"[INFO] Fehlerhafte Links gespeichert unter: {FAILED_PATH}")

[INFO] Fehlerhafte HTML-Dateien: 0
[INFO] Fehlerhafte Links gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikel_links_failed.csv


In [8]:
# Log-Datei in DataFrame umwandeln und als CSV-Datei exportieren
pd.DataFrame(log_list).to_csv(LOG_PATH, index=False)
print(f"[INFO] Erfolgreich verarbeitete HTML-Dateien: {len(log_list)}")
print(f"[INFO] Log-Datei gespeichert unter: {LOG_PATH}")

[INFO] Erfolgreich verarbeitete HTML-Dateien: 33879
[INFO] Log-Datei gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikel_links_log.csv
